# <font color=#8B4513 size=100 face="標楷體"> 謝榮哥,帆哥 </font>

# <font color=#8B4513 size=100 face="標楷體"> 資策會網路有夠爛 </font>

# <font color=#8B4513 size=100 face="標楷體"> 信用卡FB粉專_老狐狸的信用卡日記 </font>

In [788]:
import pandas as pd
import re, time, requests, datetime, gc, os, sys
from selenium import webdriver
from bs4 import BeautifulSoup

In [789]:
def 垃圾視窗NotNow():
    try:
        driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    except:
#         print(sys.exc_info())
        time.sleep(0.1)

In [790]:
# 截取塗鴉墻上貼文的連結
def fuck_Wall_PostLink(i):
    Link = 'https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]
    return Link
#截取塗鴉墻上貼文的發佈時間
def fuck_Wall_PostTime(i):
    try:
        Time = i.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = '沒發文時間阿肏你的'
    return Time
# 截取塗鴉墻上貼文的留言數
def fuck_Wall_PostComments(i):
    try:
        Comments = i.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in Comments:
            Comments = int(float(Comments.split('K')[0])*1000)
        else:
            Comments = int(Comments)
    except:
        Comments = 0
    return Comments
def CarwlList(urls, n, Posts):
    CheckList = pd.DataFrame(columns = ['Link','Time','Comments']) 

    driver.get(url)        
    for i in range(n):
        try:
            time.sleep(1)
            driver.find_element_by_css_selector('a.pam.uiBoxLightblue.uiMorePagerPrimary').click() # 加載下一頁喔糙
        except:
            time.sleep(1)
        #會跳出強迫登入的畫面喔肏
        垃圾視窗NotNow()
        soup = BeautifulSoup(driver.page_source)
        for i in soup.find_all('div', {'class':'_5pcr userContentWrapper'}):
            CheckList = pd.concat([CheckList, pd.DataFrame(data = [{'Link':fuck_Wall_PostLink(i),
                                                                    'Time':fuck_Wall_PostTime(i),
                                                                    'Comments':fuck_Wall_PostComments(i)}],
                                                           columns = ['Link','Time','Comments'])],
                                   ignore_index = True)
    #襙有重複問題為啥啊?
    CarwlList = pd.merge(left = CheckList,
                         right = Posts.loc[:,['Link','Comments']],
                         how='left',
                         on='Link',
                         suffixes=('_c', '_p'),
                         indicator=True)
    CarwlList = pd.concat([CarwlList.loc[CarwlList._merge == 'left_only', :], # 新貼文
                           CarwlList.loc[CarwlList.Comments_c > CarwlList.Comments_p, :]], # 舊貼文但有新留言
                          ignore_index=True)
    CarwlList = CarwlList.drop_duplicates(subset = 'Link',
                                          keep= 'first',
                                          inplace = False)
    return CarwlList

# <font color=#8B4513 size=100 face="標楷體"> 彈窗 </font>

In [791]:
# 關閉彈窗
def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    垃圾視窗NotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(1)
    垃圾視窗NotNow()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    垃圾視窗NotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    垃圾視窗NotNow()



# <font color=#8B4513 size=100 face="標楷體"> 展開留言 </font>


In [792]:
def open_comment():
    time.sleep(1)
    k = len(driver.find_elements_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]'))
    if k !=0:
        #這邊似乎FB不是很穩,往下拉兩次
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        垃圾視窗NotNow()
        try:
            driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
        except:
            print('fuckyou')
    time.sleep(1)


def ClickOldest():
    time.sleep(1)
    driver.find_element_by_xpath('//a[@data-testid="UFI2ViewOptionsSelector/link"]').click()
    time.sleep(1)
    try:
        driver.find_element_by_partial_link_text('Comments shown in chronological order with the newest comments at the top.').click()
    except:
        try:
            driver.find_element_by_partial_link_text('New comments and those with new replies appear at the top.').click()
        except:
            print('Plz, Check this post arragne type!')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    垃圾視窗NotNow()
    time.sleep(1)
def MoreComments():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        print('展開夠多留言： ' + str(l) + ' 次')
        try:
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            垃圾視窗NotNow()
            driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]').click()
            time.sleep(1)
            k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))
        except:
            time.sleep(0.1)
        finally:
            l += 1
    time.sleep(1)
# 偵測是否有「更多回覆」（第二層），若有則點擊
def MoreReplies():
    k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 300):
        l += 1
        print('加載： ' + str(l) + ' 次')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))
    time.sleep(1)
# 偵測是否有「展開留言」（第二層留言內容），若有則點擊
def SeeMore():
    k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    # 卡住的次數超過100次就停止
    l = 0
    while (k != 0) & (l <= 1000):
        l += 1
        print('Click See More ： ' + str(l) + ' times.')
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        driver.find_element_by_xpath('//a[@class="_5v47 fss"]').click()
        time.sleep(0.5)
        k = len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))
    time.sleep(1)
def PostExpand():
#     ClosePopup()
#     ClickOldest() # 展開留言會有無法加載更多留言的問題，暫時先不使用
    open_comment()
    MoreComments()
    MoreReplies()
    SeeMore()

# 關閉彈窗
def ClosePopup():
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(1)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    垃圾視窗NotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(1)
    垃圾視窗NotNow()
    time.sleep(0.5)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    垃圾視窗NotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    垃圾視窗NotNow()


# <font color=#8B4513 size=100 face="標楷體"> 貼文時間與內容 </font>

In [793]:
# 貼文留言數
def Post_Name(userContent):
    return userContent.find('img').attrs['aria-label']

# 貼文ID?
def Post_ID(userContent):
    return userContent.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?ref')[0].split('https://www.facebook.com/')[1]
# 貼文留言數
def Post_Comments(userContent):
    try:
        CommentCounts = userContent.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0 
    return CommentCounts
# 貼文時間
def Post_Time(userContent):
    try:
        Time = userContent.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = "Error, Please check this post's condiction!" 
    return Time

# 貼文內容
def Post_Content(userContent):
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = "There's No Text Content!"
    return Content
# 文章內容與互動摘要
def PostInfo(soup):
    # 貼文區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    PostContent = pd.DataFrame(data = [{'Name':Post_Name(userContent),
                                        'ID':Post_ID(userContent),
                                        'Time':Post_Time(userContent),
                                        'Content':Post_Content(userContent),
                                        'Comments':Post_Comments(userContent),
                                        'Link':i}],
                            columns = ['Name', 'ID', 'Time', 'Content', 'Comments', 'Link'])
    return PostContent


        

# <font color=#8B4513 size=100 face="標楷體"> 留言內容 </font>

In [794]:
def Comment_Content(element):
    try:
        Content = element.find('span', {'dir':'ltr'}).text
    except:
        Content = 'img'
    return Content
# def CommentsInfo(soup):  
#     PostComments = pd.DataFrame()
#     userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
#     try:
#         for i in userContent.select('ul._7a9a > li'):
#             # 先抓留言並放在Comment
#             Comment = pd.DataFrame(data=[{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
#                                           'Name':i.find('img').attrs['alt'],
#                                           'Time':datetime.datetime.strptime(i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
#                                           'Content':Comment_Content(i),
#                                           'RepID':userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1],
#                                           'RepName':userContent.find('img').attrs['aria-label'],
#                                           'Link':driver.current_url}],
#                                    columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
#             PostComments = pd.concat([PostComments, Comment], ignore_index=True)
#             # 留言的留言
#             for j in i.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
#                 Comment = pd.DataFrame(data=[{'ID':j.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
#                                               'Name':j.find('img').attrs['alt'],
#                                               'Time':datetime.datetime.strptime(j.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
#                                               'Content':Comment_Content(j),
#                                               'RepID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
#                                               'RepName':i.find('img').attrs['alt'],
#                                               'Link':driver.current_url}],
#                                        columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
#                 PostComments = pd.concat([PostComments, Comment], ignore_index=True)
#     except:
#         print('抓完囉!')
#     return PostComments
def 操你的留言(soup):  
    PostComments = pd.DataFrame()
    留言的留言 = soup.find('div', {'class':'_5pcr userContentWrapper'})
    try:
        for 靠 in 留言的留言:
            PostComments = pd.DataFrame(data=[{'Content': ','.join(幹.text for 幹 in 靠.select('ul[class="_7a9a"] span[class="_3l3x"]')),
                                         'Time':'隨便啦',
                                         'ID':'你猜',
                                         'RepID':'不重要',
                                         'RepName':'嘿嘿',
                                         'Name':'SKT',
                                         'Link':i
                                         }],                         
                                   columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])





#             PostComments = pd.concat([PostComments, Comment], ignore_index=True)

    except:
         print('我是遊覽器我是廢物')
    return PostComments

## <table><tr><td bgcolor=orange> 以上是定義區</td></tr></table>

# <font color=#FF0000 size=100 face="標楷體"> 分隔線--------------------------------- </font>

## <table><tr><td bgcolor=orange> 以下是執行區</td></tr></table>

# <font color=#8B4513 size=100 face="標楷體"> excel_post_Comments </font>

In [795]:
try:
    Posts = pd.read_excel('C:\\Users\\Big data\\Desktop\\class\\web_crawler\\FB\\F1B.xls')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'Comments', 'Link'])
try:
    Comments = pd.read_pickle('C:\\Users\\Big data\\Desktop\\class\\web_crawler\\FB\\FB.xlsx')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])

print(Posts.shape)
print(Comments.shape)
# #可放多個連結


(0, 6)
(0, 7)


#  <font color=#8B4513 size=100 face="標楷體"> 開啟webdriver </font>

In [773]:
url = 'https://www.facebook.com/OldFoxCreditCardDiary/?__tn__=kC-R&eid=ARDLQtqgCt6qWY3Ezk4b0B3QRM6HB_PWG_3Hiq07Qh8hDK7disJv5wvn-47fI24tDjfeTmHtBhTwbUa_&hc_ref=ARS0lgzjQNPDL14cEETlM2f9o4Wwb9g6rosmuRMtyS4klPRu7V7EgknHmahX5PV-ww0&fref=nf&__xts__[0]=68.ARCFk7hahIvw8WUXeFCI7CZy0B-ez_RG9Xvr54Rpku02fG-ixMr7trpXMmK8hCSpiFA6k_mtlK34ASQPLv_FBLjgbaMJSHHgr5N3sg9_SY2RAgZb4LMhfoBR1kAHOi_ncW_ma_O4K9wcuGI84__JdfdJ4K2VeJO5CaHf4vyLBezMymaXf_7P5ODRcykYCb3zCR2Yj1Ru97gg_0ehoAuqNuvvzK1vyEMlV1_30qmjdy5UNaV25NEvcdDB9w527qre38GDRSHREErgqz_w5V55Nryc6BBXNdoIE1TxUwBnXq1paEopxk_6LcOG-8kVp5Z_1gsQia4GFp9wgGMi3SJiZ6yb_Q'
driver = webdriver.Chrome('C:\\Users\\Big data\\Desktop\\class\\web_crawler\\chromedriver\\chromedriver.exe')
driver.get('https://www.facebook.com/')
time.sleep(1)
driver.find_element_by_partial_link_text('English').click()


In [777]:
CarwlList你妹 = CarwlList(urls=urls, n=40, Posts = Posts)#n是下拉次數,次數愈多能加載的文章愈多
CarwlList你妹

,Link,Time,Comments_c,Comments_p,_merge
0,https://www.facebook.com/OldFoxCreditCardDiary...,2016-02-26 17:00,167,NaN,left_only
1,https://www.facebook.com/OldFoxCreditCardDiary...,2019-11-02 06:00,12,NaN,left_only
6,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-30 19:36,106,NaN,left_only
7,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-30 11:07,20,NaN,left_only
8,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-21 06:00,12,NaN,left_only
9,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-20 15:30,23,NaN,left_only
10,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-18 06:00,9,NaN,left_only
25,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-15 17:56,26,NaN,left_only
26,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-15 05:00,21,NaN,left_only
27,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-13 07:13,22,NaN,left_only


In [784]:
aaa=CarwlList你妹[0:50]
aaa


,Link,Time,Comments_c,Comments_p,_merge
0,https://www.facebook.com/OldFoxCreditCardDiary...,2016-02-26 17:00,167,NaN,left_only
1,https://www.facebook.com/OldFoxCreditCardDiary...,2019-11-02 06:00,12,NaN,left_only
6,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-30 19:36,106,NaN,left_only
7,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-30 11:07,20,NaN,left_only
8,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-21 06:00,12,NaN,left_only
9,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-20 15:30,23,NaN,left_only
10,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-18 06:00,9,NaN,left_only
25,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-15 17:56,26,NaN,left_only
26,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-15 05:00,21,NaN,left_only
27,https://www.facebook.com/OldFoxCreditCardDiary...,2019-10-13 07:13,22,NaN,left_only


# <font color=#8B4513 size=100 face="標楷體"> 爬資料(雙表,可擴展,目前暫定把留言與連言回復合併為一表) </font>

In [ ]:
nComments = pd.DataFrame(columns=['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
nPost = pd.DataFrame(columns=[ 'Name','ID', 'Time', 'Content', 'RepID', 'RepName', 'Link'])
for i in CarwlList你妹.Link:
    print('正在抓: ' + i)
    driver.get(i)
    try:
        PostExpand()
        print('完成!')
        try:
            soup = BeautifulSoup(driver.page_source)
            time.sleep(1)
            PostContent = PostInfo(soup)
            nPost = pd.concat([nPost, PostContent],sort=True, ignore_index=True)
            PostComments = 操你的留言(soup)
            nComments = pd.concat([nComments, PostComments], ignore_index=True)
            
  
           
        except:
            print('抓取留言出現錯誤喔傻屌A_A')
    except:
        print('這篇留言過少所以不用展開喔傻屌A_A')
    gc.collect()
    print('完成時間: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + '\n\n------------------')

正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/239089886430361
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
加載： 6 次
這篇留言過少所以不用展開喔傻屌A_A
完成時間: 2019-11-03 18:43:59

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/1007337132938962
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-03 18:44:32

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/1005660113106664/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
這篇留言過少所以不用展開喔傻屌A_A
完成時間: 2019-11-03 18:45:06

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/1005387003133975
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
完成!
完成時間: 2019-11-03 18:45:23

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/993623234310352
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
完成!
完成時間: 2019-11-03 18:46:07

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/996272127378796/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
這

加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
加載： 6 次
加載： 7 次
加載： 8 次
加載： 9 次
加載： 10 次
加載： 11 次
加載： 12 次
加載： 13 次
加載： 14 次
Click See More ： 1 times.
完成!
完成時間: 2019-11-03 18:56:37

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/924363587902984/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
加載： 6 次
加載： 7 次
Click See More ： 1 times.
完成!
完成時間: 2019-11-03 18:56:54

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/922350041437672/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
完成!
完成時間: 2019-11-03 18:57:08

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/922156078123735/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
加載： 6 次
加載： 7 次
加載： 8 次
加載： 9 次
加載： 10 次
加載： 11 次
加載： 12 次
加載： 13 次
加載： 14 次
加載： 15 次
Click See More ： 1 times.
完成!
完成時間: 2019-11-03 18:57:30

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/9178

展開夠多留言： 173 次
展開夠多留言： 174 次
展開夠多留言： 175 次
展開夠多留言： 176 次
展開夠多留言： 177 次
展開夠多留言： 178 次
展開夠多留言： 179 次
展開夠多留言： 180 次
展開夠多留言： 181 次
展開夠多留言： 182 次
展開夠多留言： 183 次
展開夠多留言： 184 次
展開夠多留言： 185 次
展開夠多留言： 186 次
展開夠多留言： 187 次
展開夠多留言： 188 次
展開夠多留言： 189 次
展開夠多留言： 190 次
展開夠多留言： 191 次
展開夠多留言： 192 次
展開夠多留言： 193 次
展開夠多留言： 194 次
展開夠多留言： 195 次
展開夠多留言： 196 次
展開夠多留言： 197 次
展開夠多留言： 198 次
展開夠多留言： 199 次
展開夠多留言： 200 次
展開夠多留言： 201 次
展開夠多留言： 202 次
展開夠多留言： 203 次
展開夠多留言： 204 次
展開夠多留言： 205 次
展開夠多留言： 206 次
展開夠多留言： 207 次
展開夠多留言： 208 次
展開夠多留言： 209 次
展開夠多留言： 210 次
展開夠多留言： 211 次
展開夠多留言： 212 次
展開夠多留言： 213 次
展開夠多留言： 214 次
展開夠多留言： 215 次
展開夠多留言： 216 次
展開夠多留言： 217 次
展開夠多留言： 218 次
展開夠多留言： 219 次
展開夠多留言： 220 次
展開夠多留言： 221 次
展開夠多留言： 222 次
展開夠多留言： 223 次
展開夠多留言： 224 次
展開夠多留言： 225 次
展開夠多留言： 226 次
展開夠多留言： 227 次
展開夠多留言： 228 次
展開夠多留言： 229 次
展開夠多留言： 230 次
展開夠多留言： 231 次
展開夠多留言： 232 次
展開夠多留言： 233 次
展開夠多留言： 234 次
展開夠多留言： 235 次
展開夠多留言： 236 次
展開夠多留言： 237 次
展開夠多留言： 238 次
展開夠多留言： 239 次
展開夠多留言： 240 次
展開夠多留言： 241 次
展開夠多留言： 242 次
展開夠多留言： 243 次
展開夠多留言

展開夠多留言： 52 次
展開夠多留言： 53 次
展開夠多留言： 54 次
展開夠多留言： 55 次
展開夠多留言： 56 次
展開夠多留言： 57 次
展開夠多留言： 58 次
展開夠多留言： 59 次
展開夠多留言： 60 次
展開夠多留言： 61 次
展開夠多留言： 62 次
展開夠多留言： 63 次
展開夠多留言： 64 次
展開夠多留言： 65 次
展開夠多留言： 66 次
展開夠多留言： 67 次
展開夠多留言： 68 次
展開夠多留言： 69 次
展開夠多留言： 70 次
展開夠多留言： 71 次
展開夠多留言： 72 次
展開夠多留言： 73 次
展開夠多留言： 74 次
展開夠多留言： 75 次
展開夠多留言： 76 次
展開夠多留言： 77 次
展開夠多留言： 78 次
展開夠多留言： 79 次
展開夠多留言： 80 次
展開夠多留言： 81 次
展開夠多留言： 82 次
展開夠多留言： 83 次
展開夠多留言： 84 次
展開夠多留言： 85 次
展開夠多留言： 86 次
展開夠多留言： 87 次
展開夠多留言： 88 次
展開夠多留言： 89 次
展開夠多留言： 90 次
展開夠多留言： 91 次
展開夠多留言： 92 次
展開夠多留言： 93 次
展開夠多留言： 94 次
展開夠多留言： 95 次
展開夠多留言： 96 次
展開夠多留言： 97 次
展開夠多留言： 98 次
展開夠多留言： 99 次
展開夠多留言： 100 次
展開夠多留言： 101 次
展開夠多留言： 102 次
展開夠多留言： 103 次
展開夠多留言： 104 次
展開夠多留言： 105 次
展開夠多留言： 106 次
展開夠多留言： 107 次
展開夠多留言： 108 次
展開夠多留言： 109 次
展開夠多留言： 110 次
展開夠多留言： 111 次
展開夠多留言： 112 次
展開夠多留言： 113 次
展開夠多留言： 114 次
展開夠多留言： 115 次
展開夠多留言： 116 次
展開夠多留言： 117 次
展開夠多留言： 118 次
展開夠多留言： 119 次
展開夠多留言： 120 次
展開夠多留言： 121 次
展開夠多留言： 122 次
展開夠多留言： 123 次
展開夠多留言： 124 次
展開夠多留言： 125 次
展開夠多留言： 126 

加載： 1 次
加載： 2 次
完成!
完成時間: 2019-11-03 19:19:30

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/733957210276957/
展開夠多留言： 0 次
完成!
完成時間: 2019-11-03 19:19:42

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/731876787151666/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
這篇留言過少所以不用展開喔傻屌A_A
完成時間: 2019-11-03 19:19:51

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/728172077522137
加載： 1 次
完成!
完成時間: 2019-11-03 19:20:01

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/730040877335257/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
完成!
完成時間: 2019-11-03 19:20:15

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/729214454084566
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
加載： 6 次
加載： 7 次
這篇留言過少所以不用展開喔傻屌A_A
完成時間: 2019-11-03 19:20:27

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129

加載： 5 次
加載： 6 次
加載： 7 次
加載： 8 次
完成!
完成時間: 2019-11-03 19:29:26

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/620267444979268/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
Click See More ： 1 times.
完成!
完成時間: 2019-11-03 19:29:41

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/619147348424611/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
加載： 6 次
加載： 7 次
加載： 8 次
Click See More ： 1 times.
完成!
完成時間: 2019-11-03 19:29:58

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/618501755155837
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
加載： 6 次
加載： 7 次
加載： 8 次
加載： 9 次
完成!
完成時間: 2019-11-03 19:30:15

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/614996245506388
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
完成!
完成時間: 2019-11-03 19:30:29

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/616413025364710
加載： 1 次


完成!
完成時間: 2019-11-03 19:45:22

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/548278542178159/
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-03 19:45:49

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/547141178958562
完成!
完成時間: 2019-11-03 19:45:56

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/544475839225096
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-03 19:46:08

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/541257486213598/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次
加載： 5 次
加載： 6 次
Click See More ： 1 times.
Click See More ： 2 times.
完成!
完成時間: 2019-11-03 19:46:25

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/539241586415188
完成!
完成時間: 2019-11-03 19:46:35

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/539124516426895/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
加載： 3 次
加載： 4 次


展開夠多留言： 0 次
加載： 1 次
加載： 2 次
這篇留言過少所以不用展開喔傻屌A_A
完成時間: 2019-11-03 19:55:04

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/483956438610370
加載： 1 次
完成!
完成時間: 2019-11-03 19:55:15

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/482928735379807
展開夠多留言： 0 次
加載： 1 次
完成!
完成時間: 2019-11-03 19:55:26

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/photos/a.129779610694723/482037778802236/
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
這篇留言過少所以不用展開喔傻屌A_A
完成時間: 2019-11-03 19:55:35

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/481269912212356
完成!
完成時間: 2019-11-03 19:55:45

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/479574069048607
展開夠多留言： 0 次
加載： 1 次
加載： 2 次
完成!
完成時間: 2019-11-03 19:55:57

------------------
正在抓: https://www.facebook.com/OldFoxCreditCardDiary/posts/478444432494904
完成!
完成時間: 2019-11-03 19:56:05

------------------
正在抓: https://www.facebook.com/OldFoxCreditCa

# <font color=#8B4513 size=100 face="標楷體"> 合併 </font>

In [664]:
fkk = pd.merge(nPost,nComments, on='Link', how='outer', indicator='indicator_column',suffixes=['_文章','_留言'])
終於=fkk.loc[:, ['Content_文章','Time_文章', 'Comments','Content_留言','Link']]
終於

,Content_文章,Time_文章,Comments,Content_留言,Link
0,美國運通新綠卡 - 全面分析 如果懶得點閱讀這篇，這裡直接重點整理一下: 1. 消費回饋變好...,2019-10-30 11:07,20.0,"沒有SSN，看到45K首刷先衝再說當初PRG也才50K而已😅,然後現在 Gold 也悄悄改成...",https://www.facebook.com/OldFoxCreditCardDiary...
1,不算秘密的秘密 - 兩年內拿超過一張 Chase Ink Preferred 今天這篇來聊聊...,2019-10-21 06:00,12.0,"請問有推薦哪家信用可來付今年剛收到的地價稅嗎？可開新卡。謝謝,Angela Chu 可以讀看...",https://www.facebook.com/OldFoxCreditCardDiary...


# <font color=#8B4513 size=100 face="標楷體"> 保存資料</font>

In [665]:
終於.to_excel('C:\\Users\\Big data\\Desktop\\class\\web_crawler\\FB\\FB粉專_老狐狸的信用卡日記.xls')

# <font color=#8B4513 size=100 face="標楷體"> 資源參考</font>

In [666]:
#合併
https://morvanzhou.github.io/tutorials/data-manipulation/np-pd/3-7-pd-merge/
http://justimchung.blogspot.com/2018/06/pandas-dataframe.html

SyntaxError: invalid syntax (<ipython-input-666-1eb1a4973449>, line 2)

In [481]:
留言的留言

<div class="_5pcr userContentWrapper" data-ft='{"tn":"-R"}' style=""><div class="_1dwg _1w_m _q7o"><div class="_4r_y"><div class="_1wbl"></div></div><div><span class="_47we _42b7"></span><div class="l_c3pyo2v0u i_c3pynyi2f clearfix"><div class="clearfix y_c3pyo2ta3"><a aria-hidden="true" class="_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe" data-ft='{"tn":"\u003C"}' href="https://www.facebook.com/OldFoxCreditCardDiary/?ref=nf&amp;hc_ref=ARTbuAU7B-eVAVBYoeX2j9OZAcOGbCBpVNZc58F4gtJufhbMg7rby_i_PuvgiOZ9gTM&amp;__xts__%5B0%5D=68.ARBq2P75I9mveCVW29_t78yFVhnzOSIY1PzAXQTmZOwXa-6AKJ2-bNpZk1V7BfwQI970cia4DrqWaI43GNQ2JdWso3S0ZQR2XW3B4Pe1o-5OiW_wY2U5SBZ4g55IeT1sHApaXJmzPUCFM1EuaOOZrmtS2lDclCUp8xbDXhhBrAQzyNnAkOrUeCJw2_4Teq-l_ThShJBEmyo5QigEJe2nwFpQzqORGs2lLkJwj2TdD35nyd8cc4qXpIvpUNV2D-J04k1IvN9GD4ZUzsq10wJWwWKBWIWMfPE_WW3fExNNxaru9lViq11VqmOxvT_85VSxdox7TtvuQ4TYBtHl7cS-K0w&amp;__tn__=%3C-R" tabindex="-1" target=""><div class="_38vo"><!-- react-mount-point-unstable --><div><img alt="" aria-label="老狐狸的信用卡日

# <font color=#8B4513 size=100 face="標楷體"> 保存資料</font>
